In [1]:
from utils.model_utils import load_model_and_tokenizer
from utils.data_utils import load_corpus
from contrastive_evaluation import eval_model_on_corpus

In [12]:
from transformers import AutoTokenizer, BertForSequenceClassification, DistilBertTokenizer, \
    DistilBertForSequenceClassification, pipeline, AutoModelForSequenceClassification, BertJapaneseTokenizer, DistilBertModel
import os
import json

In [3]:
def load_model_and_tokenizer(model_location: str, model_type: str = "", lang: str = "",
                             from_path: bool = False, return_pipeline: bool = False):
    if not from_path:  # in this case there is a yaml config

        model_loc = yaml.load(open(model_location), Loader=yaml.FullLoader)
        this_model = model_loc[model_type][lang]
        tok_path = this_model
        print(this_model)

    else:
        this_model = model_location # there is a checkpoint locally to load
        with open(os.path.join(model_location, "config.json"), "r") as fin:
            cfg = json.load(fin)
            tok_path = cfg["_name_or_path"]
    
#     if lang
#         print("ja tokenize")
#         tokenizer = DistilBertTokenizer.from_pretrained(tok_path)
#     else:
    tokenizer = AutoTokenizer.from_pretrained(tok_path, model_input_names = ['input_ids', 'attention_mask'])  # this way it autodetects the tokenizer from pretrained model
   
    model = AutoModelForSequenceClassification.from_pretrained(this_model,
                                                                    num_labels=5,
                                                                    problem_type="single_label_classification")

    if return_pipeline:
        return pipeline("text-classification", tokenizer=tokenizer, model=model)
    else:
        return tokenizer, model


In [29]:
#p = "/home/ec2-user/SageMaker/efs/sgt/compressed_models/ja_20.0_0_2021_12_30_21_19_09/checkpoint-12504/"
p = "/home/ec2-user/SageMaker/efs/sgt/compressed_models/en_15.0_63_2021_12_02_03_43_03/checkpoint-38033"

In [31]:
model = AutoModelForSequenceClassification.from_pretrained(p,
                                                                    num_labels=5,
                                                                    problem_type="single_label_classification")

In [32]:
with open(os.path.join(p, "config.json"), "r") as fin:
            cfg = json.load(fin)
            tok_path = cfg["_name_or_path"]

In [33]:
cfg

{'_name_or_path': '/home/ec2-user/SageMaker/efs/sgt/compressed_models/en_15.0_63_2021_12_02_03_43_03/checkpoint-23445',
 'activation': 'gelu',
 'architectures': ['DistilBertForSequenceClassification'],
 'attention_dropout': 0.1,
 'classification_dropout': 0.1,
 'dim': 768,
 'dropout': 0.1,
 'hidden_dim': 3072,
 'id2label': {'0': 'LABEL_0',
  '1': 'LABEL_1',
  '2': 'LABEL_2',
  '3': 'LABEL_3',
  '4': 'LABEL_4'},
 'initializer_range': 0.02,
 'label2id': {'LABEL_0': 0,
  'LABEL_1': 1,
  'LABEL_2': 2,
  'LABEL_3': 3,
  'LABEL_4': 4},
 'max_position_embeddings': 512,
 'model_type': 'distilbert',
 'n_heads': 12,
 'n_layers': 6,
 'output_past': True,
 'pad_token_id': 0,
 'problem_type': 'single_label_classification',
 'qa_dropout': 0.1,
 'seq_classif_dropout': 0.2,
 'sinusoidal_pos_embds': False,
 'tie_weights_': True,
 'torch_dtype': 'float32',
 'transformers_version': '4.11.3',
 'vocab_size': 28996}

In [30]:
classifier_pipeline = load_model_and_tokenizer(p, from_path=True, return_pipeline=True)

404 Client Error: Not Found for url: https://huggingface.co//home/ec2-user/SageMaker/efs/sgt/compressed_models/en_15.0_63_2021_12_02_03_43_03/checkpoint-23445/resolve/main/config.json


OSError: Can't load config for '/home/ec2-user/SageMaker/efs/sgt/compressed_models/en_15.0_63_2021_12_02_03_43_03/checkpoint-23445'. Make sure that:

- '/home/ec2-user/SageMaker/efs/sgt/compressed_models/en_15.0_63_2021_12_02_03_43_03/checkpoint-23445' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/home/ec2-user/SageMaker/efs/sgt/compressed_models/en_15.0_63_2021_12_02_03_43_03/checkpoint-23445' is the correct path to a directory containing a config.json file



In [6]:
test_corpus = load_corpus("ja")

In [7]:
x = classifier_pipeline.model

In [16]:
x.base_model_prefix == 'distilbert'

True

In [24]:
y = classifier_pipeline.tokenizer

In [28]:
type(y) == BertJapaneseTokenizer

True

In [38]:
bias2results, emo2bias2results = eval_model_on_corpus(classifier_pipeline, test_corpus)

KeyboardInterrupt: 